# Functions and Relations

The library provides a set of classes to facilitate the definition of complex functions. These functions are mainly used to set up linear and nonlinear optimization functions, where derivative information or symbolic input is relevant. Also, they aim to provide a unified way to feed other classes and functions in the library.

## Functions

The most important class is the `Function` class. It allows for symbolic and numerical definition of functions, where derivative information is important. For instance, symbolic definition of a function looks like this:

In [64]:
from sigmaepsilon.math.function import Function
import sympy as sy

x1, x2, x3, x4 = variables = sy.symbols(['x1', 'x2', 'x3', 'x4'])
f = Function(3*x1 + 9*x3 + x2 + x4, variables=variables)

After defining the function, the first and second derivatives are determined automatically, and the szmbolic expressions are turned into high-performance NumPy functions via SymPy. You can evaluate the function, its gradient and Hessian as well.

In [65]:
f([0, 6, 0, 4])

10

To evaluate the gradient, call the `g` method of the instance.

In [66]:
f.g([0, 6, 0, 4])

array([3, 1, 9, 1])

To evaluate the Hessian, call the `G` method (since the function is linear, the Hessian is zero now).

In [67]:
f.G([0, 6, 0, 4])

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

To define the same function numerically, we feed the `Function` class with custom function implementations.

In [68]:
import numpy as np

# implementation of the function f(x, y) = x^2 + y
def f0(x, y):
    return x**2 + y

# implementation of the gradient of f(x, y)
def f1(x, y):
    return np.array([2*x, 1])

# implementation of the hessian of f(x, y)
def f2(x, y):
    return np.array([[0, 0], [0, 0]])

f = Function(f0, f1, f2, dimension=2)

If the function instance is defined by custom callables, the way the arguments have to be passed depends on those callables. In the previous case, the functions `f0`, `f1` and `f2` expect two arguments, hence the inputs must be provided as such.

In [69]:
f(0, 0), f.g(0, 0), f.G(0, 0)

(0,
 array([0, 1]),
 array([[0, 0],
        [0, 0]]))

Let's take a look at the next code block, where we reimplement the same function, with the exception that the callables now expect an iterable as the input instead of the coordinates separately.

In [70]:
f0 = lambda x: x[0] ** 2 + x[1]
f1 = lambda x: np.array([2*x[0], 1])
f2 = lambda x: np.array([[0, 0], [0, 0]])
f = Function(f0, f1, f2, dimension=2)

Consequently, the instance has to be called with vectorized inputs.

In [71]:
f([0, 0]), f.g([0, 0]), f.G([0, 0])

(0,
 array([0, 1]),
 array([[0, 0],
        [0, 0]]))

## Relations

Relations are like functions, but they either express an equality or an inequality of some sort and the are mainly used to express constraints when dealing with mathematical programming problems.

In [72]:
from sigmaepsilon.math.function import Equality, InEquality, Relation

In the following block, you see examples to create a computational representation of the equality

$$
x_1 + 2 x_3 + x_4 = 4
$$

In [73]:
x1, x2, x3, x4 = variables = sy.symbols(['x1', 'x2', 'x3', 'x4'])
eq = Equality(lambda x: x[0] + 2*x[2] + x[3] - 4)
eq = Equality(x1 + 2*x3 + x4 - 4, variables=variables)
eq = Equality("x1 + 2*x3 + x4 = 4", variables=variables)

Of course, symbolic definitions have the advantage of the gradient and Hessian of the input expression being calculated automatically.

Both the `Equality` and the `Inequality` classes are subclasses of `Function`, hence they can be called similarly.

In [74]:
eq([0, 0, 0, 0]), eq.g([0, 0, 0, 0]), eq.G([0, 0, 0, 0])

(-4,
 array([1, 0, 2, 1]),
 array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]))

Relations can also be related. The `relate` method returns `True` if the relation is satisfied and `False` if they are not.

In [75]:
eq.relate([4, 0, 0, 0]), eq.relate([0, 0, 0, 0])

(True, False)

The definition and usage of inequalities is very similar, except that now you also have to specify the operator at instantiation.

In [76]:
x, y = variables = sy.symbols("x y")

gt = InEquality('x + y', op='>', variables=variables)
ge = InEquality('x + y >= 0', variables=variables)
le = InEquality('x + y', op=lambda x, y: x <= y, variables=variables)
lt = InEquality('x + y', op=lambda x, y: x < y, variables=variables)

print(gt.relate([0, 0]))
print(ge.relate([0, 0]))
print(le.relate([0, 0]))
print(lt.relate([0, 0]))

False
True
True
False


From version 2.0, you don't necessarily have to use the classes `Equality` and `Inequality`, as the `Relation` class is able to decide the type itself.

In [77]:
ieq = Relation('x + y >= 2', variables=variables)
eq = Relation('x + y = 2', variables=variables)

print(type(ieq))
print(type(eq))

<class 'sigmaepsilon.math.function.relation.InEquality'>
<class 'sigmaepsilon.math.function.relation.Equality'>
